In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate, add, BatchNormalization, Activation
import os
import glob
import rasterio

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
def conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)

    return conv2

In [ ]:
def residual_conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)
        
    #skip connection    
    shortcut = Conv2D(filters, kernel_size=(1, 1), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        shortcut = BatchNormalization(axis=3)(shortcut)
    shortcut = Activation("relu")(shortcut)
    respath = add([shortcut, conv2])       
    return respath

In [ ]:
def dense_block(inputs, num_filters):
    conv1 = conv_block(inputs, num_filters)
    concat = Concatenate()([inputs, conv1])
    return concat

In [ ]:
def residual_unet(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = residual_conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = residual_conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = residual_conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = residual_conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def dense_unet(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = dense_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = dense_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = dense_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = dense_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def load_data(input_dir, mask_dir):
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')))
    # print(input_files)
    mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.tif')))
    # print(mask_files)
    images = []
    masks = []

    for input_file, mask_file in zip(input_files, mask_files):
        # Read input file
        with rasterio.open(input_file) as src:
            img = src.read(1)  # Read the first band assuming it's a single-band image
            # print(img)
            images.append(img)

        # Read mask file
        with rasterio.open(mask_file) as src:
            msk = src.read(1)  # Read the first band assuming it's a single-band image
            masks.append(msk)

    # print(np.array(masks)/255)
    return np.array(images), np.array(masks)/255

In [ ]:
def predict(model, images, masks, output_dir, model_type, number=-1):

    i=0
    for arbitrary_img, arbitrary_mask in zip(images, masks):

    # arbitrary_image_path = 'data_new\subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1024.tif'
    # arbitrary_image_mask = 'output_tif\subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1024_new_mask.tif'

    # # Read the arbitrary image
    # with rasterio.open(arbitrary_image_path) as src:
    #     arbitrary_img = src.read(1)  # Read the first band assuming it's a single-band image

    # with rasterio.open(arbitrary_image_mask) as src:
    #     arbitrary_mask = src.read(1)

    # Preprocess the arbitrary image if necessary (e.g., normalize, resize) to match the input dimensions of your model
    # ...

        predicted_mask = model.predict(np.expand_dims(arbitrary_img, axis=0))[0]

        # Threshold the predicted mask if necessary
        predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

        # Plot the arbitrary image and the predicted mask
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(arbitrary_img, cmap='gray')
        plt.title('Arbitrary Image')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(arbitrary_mask, cmap='gray')
        plt.title('Actual Mask')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(predicted_mask, cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

        plt.savefig(f"./{output_dir}/{model_type}_{i}.jpg")

        i+=1

        if (i==number):
            break

In [ ]:
input_dir = './data_new/'
mask_dir = './output_tif/'
output_dir = './outputs_diva_gis'

# Load data
images, masks = load_data(input_dir, mask_dir)

# Define input shape and number of classes
images = np.expand_dims(images, axis=-1)
input_shape = images.shape[1:]

# images_train, images_test, masks_train, masks_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Create and compile the model

# for layer in model.layers:
#     print(layer.output_shape)

In [ ]:
model = dense_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
model.fit(images, masks, validation_split=0.1, epochs=10, batch_size=2)  # Adjust epochs and batch size as needed
model.save('UNet_dense_10_epoch.h5')

In [ ]:
model = load_model("UNet_dense_10_epoch.h5")
predict(model, images, masks, output_dir, 'dense')

Epoch 1/10

568/568 [==============================] - 331s 579ms/step - loss: 0.5515 - accuracy: 0.7403 - val_loss: 0.5764 - val_accuracy: 0.7228

Epoch 2/10

568/568 [==============================] - 344s 606ms/step - loss: 0.5334 - accuracy: 0.7527 - val_loss: 0.5170 - val_accuracy: 0.7757

Epoch 3/10

568/568 [==============================] - 351s 617ms/step - loss: 0.5159 - accuracy: 0.7560 - val_loss: 0.5306 - val_accuracy: 0.7759

Epoch 4/10

568/568 [==============================] - 335s 590ms/step - loss: 0.5057 - accuracy: 0.7621 - val_loss: 0.5323 - val_accuracy: 0.7616

Epoch 5/10

568/568 [==============================] - 334s 588ms/step - loss: 0.5023 - accuracy: 0.7655 - val_loss: 0.5437 - val_accuracy: 0.7763

Epoch 6/10

568/568 [==============================] - 333s 586ms/step - loss: 0.4993 - accuracy: 0.7655 - val_loss: 0.5482 - val_accuracy: 0.7544

Epoch 7/10

568/568 [==============================] - 333s 586ms/step - loss: 0.4963 - accuracy: 0.7690 - val_loss: 0.5347 - val_accuracy: 0.7551

Epoch 8/10

568/568 [==============================] - 333s 586ms/step - loss: 0.4924 - accuracy: 0.7702 - val_loss: 0.5344 - val_accuracy: 0.7581

Epoch 9/10

568/568 [==============================] - 333s 586ms/step - loss: 0.4940 - accuracy: 0.7701 - val_loss: 0.5120 - val_accuracy: 0.7752

Epoch 10/10

568/568 [==============================] - 334s 588ms/step - loss: 0.4926 - accuracy: 0.7695 - val_loss: 0.5155 - val_accuracy: 0.7653

In [ ]:
# model = residual_unet(input_shape)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# # Train the model
# model.fit(images, masks, validation_split=0.1, epochs=10, batch_size=2)
# model.save('UNet_residual_10_epoch.h5')

Epoch 1/10

568/568 [==============================] - 307s 512ms/step - loss: 0.5312 - accuracy: 0.7491 - val_loss: 0.6539 - val_accuracy: 0.7283

Epoch 2/10

568/568 [==============================] - 288s 508ms/step - loss: 0.5077 - accuracy: 0.7587 - val_loss: 0.6505 - val_accuracy: 0.7502

Epoch 3/10

568/568 [==============================] - 288s 508ms/step - loss: 0.5054 - accuracy: 0.7583 - val_loss: 0.5607 - val_accuracy: 0.7130

Epoch 4/10

568/568 [==============================] - 288s 508ms/step - loss: 0.4997 - accuracy: 0.7664 - val_loss: 0.5442 - val_accuracy: 0.7505

Epoch 5/10

568/568 [==============================] - 288s 507ms/step - loss: 0.4971 - accuracy: 0.7656 - val_loss: 0.5596 - val_accuracy: 0.7231

Epoch 6/10

568/568 [==============================] - 288s 508ms/step - loss: 0.4961 - accuracy: 0.7675 - val_loss: 0.5309 - val_accuracy: 0.7615

Epoch 7/10

568/568 [==============================] - 288s 508ms/step - loss: 0.4942 - accuracy: 0.7666 - val_loss: 0.5641 - val_accuracy: 0.7541

Epoch 8/10

568/568 [==============================] - 288s 508ms/step - loss: 0.4932 - accuracy: 0.7664 - val_loss: 0.5777 - val_accuracy: 0.7369

Epoch 9/10

568/568 [==============================] - 289s 508ms/step - loss: 0.4908 - accuracy: 0.7701 - val_loss: 0.5484 - val_accuracy: 0.7417

Epoch 10/10

568/568 [==============================] - 289s 508ms/step - loss: 0.4925 - accuracy: 0.7671 - val_loss: 0.5234 - val_accuracy: 0.7473

In [ ]:
model = load_model("UNet_residual_10_epoch.h5")
predict(model, images, masks, output_dir, 'dense')